In [1]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import numpy as np
# import pandas as pd

In [2]:
# !pwd
path='/Users/hrm/Documents/Dropbox/DSrelated/DG_datachallenge/'
# path = '/home/ubuntu/pynb/DG_datachallenge'

### Reload Model  | Re-define Model
--- best to load model def + load weights  
(oddly the model.save-ed version didn't have the trained weights...)    
https://keras.io/models/about-keras-models/

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, ZeroPadding2D
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, rmsprop


Using TensorFlow backend.


In [4]:
def cnn_model():
    model = Sequential()

    model.add(Convolution2D(64, (3, 3), padding='same', kernel_initializer='lecun_uniform',
                            input_shape=(256, 256, 3),
                            activation='relu'))
    model.add(Convolution2D(64, (3, 3), kernel_initializer='lecun_uniform',
                            activation='relu') )
    model.add(MaxPooling2D(pool_size=(2, 2),
                           #strides=(2, 2) 
                          ))
    model.add(Dropout(0.2))

    
    model.add(Convolution2D(128, (5, 5), padding='same', kernel_initializer='lecun_uniform',
                            activation='relu'))
    
    model.add(Convolution2D(128, (5, 5), kernel_initializer='lecun_uniform',
                            activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),
                           #strides=(2, 2) 
                          ))
    model.add(Dropout(0.2))

    
    model.add(Convolution2D(256, (7, 7), padding='same', kernel_initializer='lecun_uniform',
                            activation='relu'))
    
    model.add(Convolution2D(256, (7, 7), kernel_initializer='lecun_uniform',
                            activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),
                           #strides=(2, 2) 
                          ))
    model.add(Dropout(0.2))

    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
   
    model.add(Dense(2, activation='softmax'))
    return model

In [ ]:
# cnn_model().summary()

In [5]:
model = cnn_model()

In [11]:
from keras.models import Model, load_model

In [ ]:
# cnn_model0 = load_model('./model/CNNmodelNOweights_3diffconvkerns.h5')

In [12]:
model.load_weights('./model/model_v3.1a.h5')

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 127, 127, 128)     204928    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 123, 123, 128)     409728    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 61, 61, 128)       0         
__________

In [ ]:
## pydot | graphviz installation is fiddly

In [6]:
import pydot_ng as pydot

In [7]:
pydot.find_graphviz()

{'circo': '/Users/hrm/anaconda3/envs/geopandas_env_v2/bin/circo',
 'dot': '/Users/hrm/anaconda3/envs/geopandas_env_v2/bin/dot',
 'fdp': '/Users/hrm/anaconda3/envs/geopandas_env_v2/bin/fdp',
 'neato': '/Users/hrm/anaconda3/envs/geopandas_env_v2/bin/neato',
 'sfdp': '/Users/hrm/anaconda3/envs/geopandas_env_v2/bin/sfdp',
 'twopi': '/Users/hrm/anaconda3/envs/geopandas_env_v2/bin/twopi'}

In [9]:
from keras.utils import plot_model
# plot_model(cnn_model0, to_file='model_v3.1a_test.png', show_shapes=True)
plot_model(model, to_file='./imgs/model_v3_descript.png', show_shapes=True)

In [14]:
# from keras.models import Model

from keras.layers import Input
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input


In [15]:
VGG16().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [16]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

In [17]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

# and a logistic layer -- N classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
VGG_model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False


In [18]:
VGG_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [20]:
from keras.utils import plot_model
plot_model(VGG_model, to_file='./imgs/model_v4_VGG_descript.png', show_shapes=True)